# Running instructions
Please import the function you want to test and specify the parameters for each functions explicitly before running the following tests. An example is shown below.
```
from homework_solution.py import slide_window
x = list(range(18))
width = 5
increment = 2
```
Please import function and set those arguments first before running the validation test. Please import the function before running the functional test.

# Problem: Slide window
Implement a sliding window for an arbitrary input list. The function should take the window width and the window increment as inputs and should produce a sequence of overlapping lists from the input list. For example, given `x=list(range(15))`, the following is the output given a window width of 5 and window increment of 2.
 ```
  [[0, 1, 2, 3, 4],
   [2, 3, 4, 5, 6],
   [4, 5, 6, 7, 8],
   [6, 7, 8, 9, 10],
   [8, 9, 10, 11, 12],
   [10, 11, 12, 13, 14]]
   ```
In the event that the input parameters do not yield a complete set of even sublists, just truncate the ragged tail. For example,
```
 >>> slide_window(list(range(18)),5,2)
  [[0, 1, 2, 3, 4],
   [2, 3, 4, 5, 6],
   [4, 5, 6, 7, 8],
   [6, 7, 8, 9, 10],
   [8, 9, 10, 11, 12],
   [10, 11, 12, 13, 14],
   [12, 13, 14, 15, 16]]
 ```
Here is the function signature: slide_window(x,width,increment) where increment>0, width>0, and x is a list.


**validation Test** <br> 

In [ ]:
assert isinstance(x, list), "x must be a list"
assert isinstance(width, int) and width>0, "width must be an integer and greater than 0"
assert isinstance(increment, int) and increment>0, "increment must be an integer and greater than 0"

**Function Test**<br>

In [ ]:
# test cases when input parameters don't yield a complete set of even sublists
x = [1,2,3,4,5,6,7,8,9,10,11,12]
width = 3
increment = 4
assert slide_window(x,width,increment) == [[1,2,3],[5,6,7],[9,10,11]], "Checking cases input parameters can't yield complete set of even sublists"

In [ ]:
# test cases when input parameters generate complete set of sublists
x=[1,2,3,4,5,6,7,8,9,10,11]
width =5 
increment =2
assert slide_window(x,width,increment) == [[1, 2, 3, 4, 5], [3, 4, 5, 6, 7], [5, 6, 7, 8, 9], [7, 8, 9, 10, 11]], "Checking cases input parameters yield complete set of even sublists"

# Problem: Duration Tracker Generators
The code below defines a generator that returns the duration of its lifetime when called.
```
from time import sleep
import random
from datetime import datetime
import itertools as it

def producer():
    'produce timestamps'
    starttime = datetime.now()
    while True:
        sleep(random.uniform(0,0.2))
        yield datetime.now()-starttime
```
For example,
```
>>> p = producer()
>>> next(p)
datetime.timedelta(0, 0, 106641)
 ```
Note that the output of producer has a seconds attribute. Write a generator that tracks the output of this producer and ultimately returns the number of odd numbered seconds that have been iterated over. The usage pattern is the following,
```
>>> t = tracker(p,limit=2)
>>> next(t)
1
>>> list( tracker(p,limit=2))
[1,2] 
The limit keyword argument is the number of odd-numbered seconds to track until completion.

>>> list( tracker(p,limit=5))
[0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5] 
```
The last line is interesting because is shows that the producer's seconds value output was an even number for the first six iterations. Your tracker generator should also receive input that changes the existing limit,
```
>>> t = tracker(p,limit=3)
>>> next(t)
0
>>> next(t)
0
>>> t.send(5)
1
>>> list(t)
[1, 1, 1, 1, 2, 3, 4, 5]
```

Since the random.uniform will generate different values for each run. Thus we choose to define the following producers to help the test so we can control the sleep time explicitly. 

In [1]:
# import your producer and tracker
#from hw import producer, tracker
from time import sleep
from datetime import datetime
# define dummpy producer so we can control the sleep time
def constant_producer(i):
    starttime = datetime.now()
    while True:
        sleep(i)
        yield datetime.now() - starttime

def increment_producer():
    starttime = datetime.now()
    i = 1
    while True:
        sleep(i)
        i+=1
        yield datetime.now() - starttime

##### In this problem, since some of the parameter names are not explicitly stated in the original problem. Thus we used the following names.
function call: `tracker(p, limits)`

send method: `tracker.send(new_limits)`

**Validation Tests** <br>
Check the limits passed to `tracker` and new_limits passed to `send` method.

In [ ]:
from types import GeneratorType

assert isinstance(p, GeneratorType), "input producer must be generator type"
assert isinstance(limits, int) and limits >0, "limits must be integer and greater than 0"
assert isinstance(new_limits, int) and new_limits >0, "the parameter passed to send must be integer and greater than 0"

**Functional Tests** <br>
Test the functionality with cosutomed producer so we can control the sleep time

In [ ]:
# test tracker for producer with different sleep time 
# notice that when the constant time is 2n, the tracker will run forever
assert list(tracker(constant_producer(1), 2)) == [1, 1, 2], "constant 1 sleep time test"
assert list(tracker(increment_producer(), 2))== [1, 2], "increment sleep time test"

In [ ]:
# test tracker with different number of limits
assert list(tracker(constant_producer(1), 1)) == [1], "const 1 sleep time and limits=1"
assert list(tracker(constant_producer(1), 3)) == [1, 1, 2, 2, 3], "constant 1 sleep time and limits=3"

In [ ]:
# test tracker with send
# send limits>3
t = tracker(constant_producer(1), limits=3)
assert next(t) == 1, "limits=3<new_limits=5"
assert t.send(5) == 1, "limits=3<new_limits=5"
assert list(t) == [2, 2, 3, 3, 4, 4, 5], "limits=3<new_limits=5"

# send limits<3
t = tracker(constant_producer(1), limits=3)
assert next(t) == 1, "limits=3>new_limits=2"
assert t.send(2) == 1, "limits=3>new_limits=2"
assert list(t) == [2], "limits=3>new_limits=2"

# send limits<3 and when the tracker has alreay reach number>=3
t = tracker(constant_producer(1), limits=3)
assert next(t) == 1, "limits=3>new_limits=2 and when we send the message the tracker has reached iteration=3"
assert next(t) == 1, "limits=3>new_limits=2 and when we send the message the tracker has reached iteration=3"
assert t.send(2) == 2, "limits=3>new_limits=2 and when we send the message the tracker has reached iteration=3"
assert list(t) == [], "limits=3>new_limits=2 and when we send the message the tracker has reached iteration=3"

## Problem: Fibonacci
The Fibonacci numbers are defined by the following recursion: `F[n] = F[n-1]+F[n-2]` with initial values `F[1]=F[0]=1`. Write a generator to compute the first n Fibonacci numbers. For example, for n=10, the output for `list(fibonacci(n))` should be `[1,1,2,3,5,8,13,21,34,55]`.


##### Validation testcases

In [2]:
assert isinstance(n, int) and n>0, "n must be integer and greater than 0"
# Check if function is of generator type
import types
f = fibonacci(5)
assert isinstance(f, types.GeneratorType), "fibonacci function must be implemented as generator"

##### Functional testcases

In [4]:
# Check for first 10 elements
assert list(fibonacci(10)) == [1, 1, 2, 3, 5, 8, 13, 21, 34, 55], "test fibonacci(10)->case n>1"
assert list(fibonacci(1)) == [1], "test fibonacci(1)->case n=1"